In [1]:
import os
import cv2
import numpy as np
from network import *
from keras.models import load_model

# PRETRAIN MODE

In [3]:
base_model =  Network().YOLO_v1_pretrain()

Metal device set to: Apple M1


2023-04-01 17:50:30.417804: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-04-01 17:50:30.418636: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [4]:
print(base_model.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 backbone (Sequential)       (None, 7, 7, 1024)        22439488  
                                                                 
 average_pooling2d (AverageP  (None, 3, 3, 1024)       0         
 ooling2D)                                                       
                                                                 
 flatten (Flatten)           (None, 9216)              0         
                                                                 
 dense (Dense)               (None, 20)                184340    
                                                                 
 activation (Activation)     (None, 20)                0         
                                                             

In [5]:
img_load = os.path.join('VOCdevkit_2007_trainval', 
                        'VOC2007', 
                        'JPEGImages', '000005.jpg')
img = cv2.cvtColor(cv2.imread(img_load), cv2.COLOR_BGR2RGB)
img = cv2.resize(img, (224, 224), interpolation=cv2.INTER_AREA)
img = np.expand_dims(img, 0)

In [6]:
res = base_model.predict(img)

2023-04-01 17:50:40.351993: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-04-01 17:50:40.458195: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 306ms/step


In [7]:
print(res.shape)

(1, 20)


In [10]:
base_model.save_weights('./pretrain_model_weights', save_format='tf')

# DETECT MODE

In [12]:
yolo_v1 = Network().YOLO_v1()

In [13]:
print(yolo_v1.summary())

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 448, 448, 3)]     0         
                                                                 
 backbone (Sequential)       (None, 14, 14, 1024)      22439488  
                                                                 
 neck (Sequential)           (None, 7, 7, 1024)        37769216  
                                                                 
 fc_layers (Sequential)      (None, 1470)              211547582 
                                                                 
Total params: 271,756,286
Trainable params: 271,729,918
Non-trainable params: 26,368
_________________________________________________________________
None


In [14]:
img_load = os.path.join('VOCdevkit_2007_trainval', 
                        'VOC2007', 
                        'JPEGImages', '000005.jpg')
img = cv2.cvtColor(cv2.imread(img_load), cv2.COLOR_BGR2RGB)
img = cv2.resize(img, (448, 448), interpolation=cv2.INTER_AREA)
img = np.expand_dims(img, 0)

In [15]:
res = yolo_v1.predict(img)

1/1 [==============================] - 0s 300ms/step


2023-04-01 17:51:36.495657: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [16]:
print(res.shape)

(1, 1470)


# Transfer Learning

In [18]:
base_model.load_weights('pretrain_model_weights')
layer_dict = dict([(layer.name, layer) for layer in base_model.layers])


In [19]:
for k in layer_dict:
    print(f'{k}: {layer_dict[k]}')

input_1: <keras.engine.input_layer.InputLayer object at 0x28a902ec0>
backbone: <keras.engine.sequential.Sequential object at 0x28aa646a0>
average_pooling2d: <keras.layers.pooling.average_pooling2d.AveragePooling2D object at 0x28a8f7bb0>
flatten: <keras.layers.reshaping.flatten.Flatten object at 0x28aac09d0>
dense: <keras.layers.core.dense.Dense object at 0x28aa66c20>
activation: <keras.layers.core.activation.Activation object at 0x28a8f7df0>


In [20]:
for layer in yolo_v1.layers:
    layer_name = layer.name
    if layer_name in layer_dict:   
        print(f"{layer_name} is loaded")
        layer.set_weights(layer_dict[layer_name].get_weights())

backbone is loaded


In [22]:
para_sum = 0
for layer in yolo_v1.layers:
    layer_name = layer.name
    if layer_name == 'backbone':
        ws = layer.get_weights()
        for w in ws:
            t = 1
            for ws in w.shape:
                t *= ws 
            para_sum += t

assert para_sum == 22439488